# Convert Runtimes from pandas-compatible .csv File to LaTeX Code

## Default Values for Papermill Parameters

In [ ]:
PARAM_RUNTIMES_PATH = "../outputs/statistics_merged_result_set.csv"
PARAM_OUT_FILENAME = "runtimes_table.tex"

## Import and Set Parameters

In [ ]:
from subroc import util

import pandas as pd
import os

# fill environment variables into params
PARAM_RUNTIMES_PATH = util.prepend_experiment_output_path(PARAM_RUNTIMES_PATH)

# get environment variables
STAGE_OUTPUT_PATH = os.environ.get("STAGE_OUTPUT_PATH", "../outputs")

## Read the Runtimes

In [ ]:
runtimes_df = pd.read_csv(PARAM_RUNTIMES_PATH)

print(runtimes_df)

## Prepare the Runtime Table Entries

In [ ]:
def runtimes_to_latex_df(with_optimistic_estimate: bool):
    oe_filtered_df = runtimes_df[runtimes_df["optimistic_estimate"] == with_optimistic_estimate]

    qf_names = {
        "average_ranking_loss": r"$\varphi_{Adult}^{rasl}$",
        "roc_auc_score": r"$\varphi_{Adult}^{ROCAUC}$",
        "prc_auc_score": r"$\varphi_{Adult}^{PRAUC}$",
    }

    df_groupby = oe_filtered_df.groupby("qf_name", as_index=False)
    latex_rows = []

    for key in df_groupby.groups.keys():
        group = df_groupby.get_group(key)
        print(group)

        qf_name = qf_names[key]
        latex_rows.append([qf_name, *[f"{group['time_mean'].iloc[i]:.4f}" + r" $\pm$ " + f"{group['time_std'].iloc[i]:.4f}" for i in range(len(group))]])

    return pd.DataFrame(latex_rows, columns=["Interestingness Measure", *[r"$\theta_{dc}=" + str(i+1) + "$" for i in range(len(latex_rows[0])-1)]])

with_oe_runtime_latex_df = runtimes_to_latex_df(True)
with_oe_runtime_latex_df.to_latex(f"{STAGE_OUTPUT_PATH}/with_oe_{PARAM_OUT_FILENAME}", float_format="%.4f", index=False)

no_oe_runtime_latex_df = runtimes_to_latex_df(False)
no_oe_runtime_latex_df.to_latex(f"{STAGE_OUTPUT_PATH}/no_oe_{PARAM_OUT_FILENAME}", float_format="%.4f", index=False)